In [1]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')

In [27]:
electric = pd.read_csv("Full Eletric Interval 042016.csv")

In [28]:
electric['Datetime'] = pd.to_datetime(electric['Date'] + ' ' + electric['Start Time'])

In [4]:
propertyCode = pd.read_excel("SIMS_Property_List_4-6-2016_Revised.xlsx")

In [45]:
weather = pd.read_csv('weather_prepared_final.csv')

In [48]:
weather.DATE = pd.to_datetime(weather.DATE)

### List of codes

In [5]:
codelist = [b[0] for b in electric['Meter'].str.split()]

In [6]:
codes = pd.Series(codelist)

### Usage by day

In [30]:
electric_date_full = electric
electric_date_full.Datetime = electric_date_full.Datetime.dt.date

In [37]:
electric_day = electric_date_full.drop('Temperature', 1).groupby(by=['Meter', 'Datetime']).sum()

### Temperature

In [63]:
temperature = weather.drop(['HUMIDITY', 'LON', 'LAT', 'STATION'], 1).iloc[1::4, :]

### Daily average temperature

In [73]:
daily_temperature = temperature
daily_temperature.DATE = temperature.DATE.dt.date
daily_temperature = daily_temperature.groupby(by='DATE').mean()

In [265]:
def normal_usage(meter, date, temperature):
    weekend = pd.to_datetime(date).weekday in [5,6]
    meter_usage = electric_day.ix[meter]
    return meter_usage[(daily_temperature.ix[meter_usage.index]['TEMPERATURE'] > temperature - 5) & (daily_temperature.ix[meter_usage.index] < temperature + 5).squeeze() & (abs(pd.to_datetime(date).dayofyear - pd.to_datetime(meter_usage.index).dayofyear) < 50) & (pd.to_datetime(meter_usage.index).weekday.isin([5,6]) == weekend)]['Usage'].mean()

In [269]:
def percent_difference(meter, date, temperature, usage):
    normal = normal_usage(meter, date, temperature)
    return abs((normal - usage) / normal)